In [1]:
import pandas as pd
sms = pd.read_csv('spam_or_not_spam.csv')
sms.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [2]:
len(sms)

3000

In [3]:
sms.label.sum()

500

In [9]:
sms = sms.dropna()

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf_model.fit_transform(raw_documents=sms.email).toarray()

In [11]:
tfidf_docs.shape

(2999, 34079)

In [12]:
sms.label.sum()

499

In [13]:
mask = sms.label.astype(bool).values
spam_centroid = tfidf_docs[mask].mean(axis=0)
ham_centroid = tfidf_docs[~mask].mean(axis=0)
spam_centroid.round(2)

array([0.  , 0.  , 0.01, ..., 0.  , 0.  , 0.  ])

In [14]:
ham_centroid.round(2)

array([0.01, 0.  , 0.01, ..., 0.  , 0.  , 0.  ])

In [15]:
spamminess_score = tfidf_docs.dot(spam_centroid - ham_centroid)
spamminess_score.round(2)

array([-0.02,  0.  ,  0.  , ...,  0.07,  0.04,  0.01])

In [18]:
from sklearn.preprocessing import MinMaxScaler
sms['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))
sms['lda_predict'] = (sms.lda_score > .5).astype(int)
sms['label lda_predict lda_score'.split()].round(2).tail(15)

,label,lda_predict,lda_score
2985,1,0,0.32
2986,1,1,0.91
2987,1,1,0.77
2988,1,0,0.37
2989,1,0,0.46
2990,1,1,0.62
2991,1,1,0.62
2992,1,0,0.40
2993,1,0,0.49
2994,1,0,0.48


In [20]:
(1. - (sms.label - sms.lda_predict).abs().sum() / len(sms)).round(3)

0.932

In [25]:
from sklearn.metrics import confusion_matrix

In [27]:
confusion_matrix(sms['label'], sms['lda_predict'])

array([[2500,    0],
       [ 205,  294]], dtype=int64)

In [30]:
import pandas as pd
pd.options.display.width=120
sms2 = pd.read_csv('spam_or_not_spam.csv')

In [32]:
sms2.head(6)

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0
5,i just had to jump in here as carbonara is on...,0


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf = TfidfVectorizer(tokenizer = casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=sms.email).toarray()
len(tfidf.vocabulary_)

34079

In [35]:
tfidf_docs.shape

(2999, 34079)

In [36]:
len(sms2)

3000

In [39]:
tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()

In [40]:
tfidf_docs.shape

(2999, 34079)

In [41]:
sms.label.sum()

499

In [42]:
from sklearn.decomposition import PCA

In [44]:
pca = PCA(n_components=16)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)

In [49]:
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns)

In [51]:
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
0,0.228,-0.102,-0.039,0.005,-0.031,-0.002,0.110,0.007,-0.024,0.045,-0.061,-0.098,-0.040,-0.001,-0.007,0.028
1,-0.058,0.091,0.024,-0.058,0.031,-0.037,-0.042,-0.007,-0.043,0.234,0.114,-0.029,-0.022,-0.029,-0.083,0.024
2,-0.114,0.030,0.028,-0.113,0.011,-0.047,-0.028,-0.082,-0.008,0.184,0.025,-0.038,-0.030,-0.028,0.008,-0.018
3,-0.055,0.037,0.005,-0.053,0.016,-0.041,0.010,-0.068,-0.039,0.004,-0.035,-0.039,0.059,-0.028,-0.036,-0.004
4,-0.139,0.069,0.000,-0.029,0.050,-0.022,-0.033,0.051,-0.016,0.254,0.148,-0.011,-0.019,-0.041,-0.035,-0.001
5,-0.157,0.044,0.051,-0.064,0.056,-0.015,-0.070,0.034,0.010,0.324,0.169,-0.038,-0.050,-0.047,-0.044,0.015


In [53]:
tfidf.vocabulary_

{'date': 6778,
 'wed': 32109,
 'number': 19358,
 'aug': 2163,
 'from': 11039,
 'chris': 4839,
 'garrigues': 11312,
 'cwg': 6604,
 'dated': 6780,
 'numberfanumberd': 19590,
 'deepeddy': 6991,
 'com': 5357,
 'message': 17619,
 'id': 13424,
 'tmda': 29610,
 'vircio': 31561,
 'i': 13365,
 'can': 4175,
 't': 28800,
 'reproduce': 24812,
 'this': 29372,
 'error': 9372,
 'for': 10738,
 'me': 17445,
 'it': 14627,
 'is': 14549,
 'very': 31421,
 'repeatable': 24746,
 'like': 16324,
 'every': 9547,
 'time': 29522,
 'without': 32452,
 'fail': 9983,
 'the': 29279,
 'debug': 6896,
 'log': 16590,
 'of': 20641,
 'pick': 22132,
 'happening': 12383,
 'pick_it': 22133,
 'exec': 9660,
 'inbox': 13731,
 'list': 16423,
 'lbrace': 15972,
 'subject': 28261,
 'ftp': 11097,
 'rbrace': 24102,
 'sequence': 26397,
 'mercury': 17596,
 'ftoc_pickmsgs': 11090,
 'hit': 12861,
 'marking': 17218,
 'hits': 12872,
 'tkerror': 29593,
 'syntax': 28757,
 'in': 13710,
 'expression': 9840,
 'int': 14173,
 'note': 19252,
 'if': 

In [54]:
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(),\
                                    tfidf.vocabulary_.keys())))

In [55]:
terms

('_',
 '__',
 '___',
 '___amex',
 '___disc',
 '___i',
 '___instant',
 '___mastercard',
 '___oop',
 '___visa',
 '___x',
 '___yes',
 '__db',
 '__extremely__',
 '__getstate__',
 '__gxx_personality_vnumber',
 '__init__',
 '__ksymtab_snd_hack_usb_set_interface',
 '__number',
 '__osf__',
 '__p_',
 '__setstate__',
 '__stub',
 '_a',
 '_aaron',
 '_additional_',
 '_all',
 '_all_',
 '_and_',
 '_anil',
 '_any_',
 '_are',
 '_are_',
 '_at_',
 '_audio',
 '_awesome',
 '_bad_file_number_can',
 '_because_',
 '_bill',
 '_blank',
 '_brad',
 '_buckles_',
 '_buffy',
 '_but',
 '_chance_',
 '_charlie',
 '_clay',
 '_client_',
 '_dan',
 '_datadir',
 '_dean',
 '_demand_',
 '_did_',
 '_displays',
 '_does',
 '_duh_',
 '_elvis_',
 '_emacs',
 '_emotional_',
 '_everything_',
 '_exactly_',
 '_exit',
 '_exmh_number',
 '_exmh_numberp',
 '_extremely_',
 '_fads',
 '_for',
 '_former',
 '_fundamental_',
 '_goal',
 '_haiku_',
 '_heather',
 '_himself_',
 '_his_',
 '_i',
 '_i_',
 '_ian',
 '_included',
 '_incredibly',
 '_instal

In [56]:
weights = pd.DataFrame(pca.components_, columns=terms, index=\
                      ['topic{}'.format(i) for i in range(16)])
pd.options.display.max_columns=8
weights.head(4).round(3)

,_,__,___,___amex,...,활황을,훨씬,힘입어,ｉt的技
topic0,0.023,0.000,-0.005,0.0,...,-0.0,-0.0,-0.0,0.0
topic1,0.011,0.001,-0.000,-0.0,...,0.0,0.0,0.0,0.0
topic2,-0.008,-0.001,-0.073,0.0,...,0.0,0.0,0.0,0.0
topic3,-0.034,0.000,0.129,0.0,...,0.0,0.0,0.0,0.0


In [59]:
pd.options.display.max_columns=12
deals = weights['half off free crazy deals only '.split()].\
round(3) * 100

In [60]:
deals

,half,off,free,crazy,deals,only
topic0,-0.4,-1.0,-1.5,-0.1,-0.1,-1.9
topic1,-0.7,-1.3,0.1,-0.2,0.0,-3.6
topic2,-0.4,0.0,9.6,0.1,0.2,2.0
topic3,-0.9,-1.0,4.9,-0.2,0.1,-0.1
topic4,0.2,-0.6,-2.0,-0.0,0.0,-1.8
topic5,-0.0,-0.1,9.5,0.0,0.5,1.1
topic6,0.6,0.2,-6.5,-0.0,-0.2,-1.6
topic7,-0.5,0.9,1.6,0.1,0.0,0.9
topic8,0.2,1.0,-4.4,0.1,0.4,-1.5
topic9,0.4,-0.9,4.8,-0.2,0.1,-2.8


In [61]:
deals.T.sum()

topic0     -5.0
topic1     -5.7
topic2     11.5
topic3      2.8
topic4     -4.2
topic5     11.0
topic6     -7.5
topic7      3.0
topic8     -4.2
topic9      1.4
topic10     1.7
topic11     3.7
topic12    12.2
topic13    -2.6
topic14     1.6
topic15    11.7
dtype: float64

In [63]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=sms.email)\
                       .toarray())

In [64]:
column_names, terms = zip(*sorted(zip(counter.vocabulary_.values(),\
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms

In [68]:
sms

,email,label,lda_score,lda_predict
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,0.138859,0
1,martin a posted tassos papadopoulos the greek ...,0,0.266151,0
2,man threatens explosion in moscow thursday aug...,0,0.280394,0
3,klez the virus that won t die already the most...,0,0.250174,0
4,in adding cream to spaghetti carbonara which ...,0,0.268036,0
...,...,...,...,...
2995,abc s good morning america ranks it the NUMBE...,1,0.403589,0
2996,hyperlink hyperlink hyperlink let mortgage le...,1,0.944242,1
2997,thank you for shopping with us gifts for all ...,1,0.699741,1
2998,the famous ebay marketing e course learn to s...,1,0.478429,0


In [70]:
sms.email[0]

' date wed NUMBER aug NUMBER NUMBER NUMBER NUMBER NUMBER from chris garrigues cwg dated NUMBER NUMBERfaNUMBERd deepeddy com message id NUMBER NUMBER tmda deepeddy vircio com i can t reproduce this error for me it is very repeatable like every time without fail this is the debug log of the pick happening NUMBER NUMBER NUMBER pick_it exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER exec pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER NUMBER NUMBER ftoc_pickmsgs NUMBER hit NUMBER NUMBER NUMBER marking NUMBER hits NUMBER NUMBER NUMBER tkerror syntax error in expression int note if i run the pick command by hand delta pick inbox list lbrace lbrace subject ftp rbrace rbrace NUMBER NUMBER sequence mercury NUMBER hit that s where the NUMBER hit comes from obviously the version of nmh i m using is delta pick version pick nmh NUMBER NUMBER NUMBER compiled on URL at sun mar NUMBER NUMBER 

In [72]:
bow_docs.email[0][bow_docs.email[0] > 0]

array([], dtype=int64)

In [73]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)
ldia.components_.shape

(16, 34079)

In [74]:
components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)

In [75]:
components.round(2).head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
_,7.94,0.06,160.17,45.22,0.06,12.58,...,0.06,0.06,3.80,0.06,0.06,0.06
__,2.48,0.06,3.57,0.06,0.06,1.14,...,0.06,0.06,0.06,0.06,0.06,0.06
___,0.06,249.32,18.86,191.25,0.06,205.81,...,0.06,0.71,2.16,0.06,0.06,0.06


In [76]:
components.topic1.sort_values(ascending=False)[:10]

number    1672.225093
rpm        895.141689
i          833.700666
list       832.017335
the        645.204416
it         454.466137
and        430.774694
a          405.550005
to         387.292274
url        358.756344
Name: topic1, dtype: float64

In [77]:
ldia16_topic_vectors = ldia.transform(bow_docs)
ldia16_topic_vectors = pd.DataFrame(ldia16_topic_vectors, columns=columns)

In [78]:
ldia16_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic10,topic11,topic12,topic13,topic14,topic15
0,0.00,0.0,0.00,0.0,0.00,0.61,...,0.0,0.00,0.0,0.00,0.0,0.0
1,0.00,0.0,0.49,0.0,0.00,0.00,...,0.0,0.51,0.0,0.00,0.0,0.0
2,0.11,0.0,0.60,0.0,0.18,0.00,...,0.0,0.10,0.0,0.00,0.0,0.0
3,0.00,0.0,0.00,0.3,0.49,0.12,...,0.0,0.00,0.0,0.08,0.0,0.0
4,0.00,0.0,0.56,0.0,0.00,0.12,...,0.0,0.31,0.0,0.00,0.0,0.0


In [79]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ldia16_topic_vectors,
                                                   sms.label, test_size=0.5,
                                                   random_state=42)

In [80]:
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)

In [81]:
sms['lda16_spam'] = lda.predict(ldia16_topic_vectors)

In [82]:
round(float(lda.score(X_test, y_test)), 2)

0.95

In [83]:
sms

,email,label,lda_score,lda_predict,lda16_spam
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,0.138859,0,0
1,martin a posted tassos papadopoulos the greek ...,0,0.266151,0,0
2,man threatens explosion in moscow thursday aug...,0,0.280394,0,0
3,klez the virus that won t die already the most...,0,0.250174,0,0
4,in adding cream to spaghetti carbonara which ...,0,0.268036,0,0
...,...,...,...,...,...
2995,abc s good morning america ranks it the NUMBE...,1,0.403589,0,1
2996,hyperlink hyperlink hyperlink let mortgage le...,1,0.944242,1,1
2997,thank you for shopping with us gifts for all ...,1,0.699741,1,1
2998,the famous ebay marketing e course learn to s...,1,0.478429,0,1


In [85]:
confusion_matrix(sms.label, sms.lda16_spam)

array([[2437,   63],
       [  78,  421]], dtype=int64)

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=sms.email).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_docs,\
                                                   sms.label.values,
                                                   test_size=0.5,
                                                   random_state=42)

In [90]:
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
round(float(lda.score(X_train, y_train)), 3)

0.999

In [91]:
round(float(lda.score(X_test, y_test)), 3)

0.618

In [92]:
ldia32 = LDiA(n_components=32, learning_method='batch')
ldia32 = ldia32.fit(bow_docs)
ldia32.components_.shape

(32, 34079)

In [96]:
ldia32_topic_vectors = ldia32.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(ldia32.n_components)]
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors, columns = columns32)

In [97]:
ldia32_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,...,topic26,topic27,topic28,topic29,topic30,topic31
0,0.00,0.0,0.14,0.0,0.00,0.00,...,0.44,0.0,0.0,0.29,0.0,0.0
1,0.04,0.0,0.00,0.0,0.00,0.00,...,0.00,0.0,0.0,0.00,0.0,0.0
2,0.00,0.0,0.20,0.0,0.17,0.06,...,0.00,0.0,0.0,0.00,0.0,0.0
3,0.00,0.0,0.86,0.0,0.00,0.07,...,0.00,0.0,0.0,0.00,0.0,0.0
4,0.00,0.0,0.22,0.0,0.00,0.00,...,0.00,0.0,0.0,0.20,0.0,0.0


In [99]:
X_train, X_test, y_train, y_test = train_test_split(ldia32_topic_vectors,\
                                                   sms.label, test_size=0.5,
                                                   random_state=42)
lda = LDA(n_components=1)
lda = lda.fit(X_train, y_train)
sms['ldia32_spam'] = lda.predict(ldia32_topic_vectors)

In [100]:
X_train.shape

(1499, 32)

In [101]:
round(float(lda.score(X_train, y_train)), 3)

0.949

In [102]:
round(float(lda.score(X_test, y_test)), 3)

0.933

In [103]:
lda = LDA(n_components=1)
lda = lda.fit(tfidf_docs, sms.label)
sms['lda_spaminess'] = lda.predict(tfidf_docs)
((sms.label - sms.lda_spaminess) ** 2.).sum() ** .5

0.0

In [105]:
from sklearn.model_selection import cross_val_score
lda = LDA(n_components=1)
scores = cross_val_score(lda, tfidf_docs, sms.label, cv=5)
"Accuracy: {:.2f} (+/-{:.2f})".format(scores.mean(), scores.std() * 2)

'Accuracy: 0.49 (+/-0.07)'